# Data Processing

In [1]:
# Load packages.
import gzip
import numpy as np
import pandas as pd
# Intialize the pandas preferences.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Print version numbers.
print('numpy', np.__version__)
print('pandas', pd.__version__)

numpy 1.22.3
pandas 1.4.2


## Helper Functions

In [2]:
# Define a function to load the tgp meta data.
def load_tgp_meta_data():
    """
    Returns the 1000 Genomes Project meta data as a pandas dataframe and dictionary.
    """
    df = pd.read_csv('../meta_data/integrated_call_samples_v3.20130502.ALL.panel', sep='\t')
    dicc = {col: df[col].values for col in ['sample', 'pop', 'super_pop']}
    dicc['idx'] = df.index.values
    return df, dicc

In [3]:
# Define a function to load the qc table.
def load_qc_table(prefix):
    """
    Returns the vcf qc table as pandas dataframe and dictionary.
    """
    df = pd.read_csv(f'../meta_data/{prefix}_qc_chr6.txt.gz', sep='\t')
    dicc = {col: df[col].values for col in df.columns.values}
    return df, dicc

In [4]:
# Define a function to load the hla pbd cds tables.
def load_hla_pbd_cds_table(prefix, suffix):
    """
    Returns the HLA PBD CDS table as a pandas dataframe.
    """
    return pd.read_csv(f'../hla_pbd_cds_tables/{prefix.upper()}full_HLASeq{suffix.upper()}.csv')

In [5]:
# Define a function to load the hla pbd cds vcf files.
def load_hla_pbd_cds_vcf(prefix):
    """
    Returns the HLA PBD CDS vcf as a pandas dataframe.
    """
    return pd.read_csv(f'../meta_data/{prefix}_hla_pbd_cds_vcf_no_header.txt.gz', sep='\t')

## Ensuring that the Chromosome 6 Exon Regions File is Properly Sorted

In [13]:
# Load the exon region file as a panadas dataframe.
exons_df = pd.read_csv(
    '../meta_data/ncbi_refseq_select_chr6_exons.txt',
    usecols=[0, 1, 2], names=['chr', 'start', 'end'], sep='\t'
)
# Sort the exons.
sorted_exons_df = exons_df.sort_values(by=['start', 'end']).reset_index(drop=True)
# Export the sorted exons regions file.
sorted_exons_df.to_csv('../meta_data/ncbi_refseq_select_chr6_exons.txt', sep='\t', index=False, header=False)

## Validating the TGP Sample Ordering in the Chromosome 6 VCF File

Sanity check that the samples in the vcf and panel file are in the same order.

In [17]:
# Intialize the path to the vcf file.
chr6_vcf = '../vcfs/ALL.chr6.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
# Read the vcf file.
with gzip.open(chr6_vcf, 'rt') as data:
    # Iterate through every line in the original vcf file.
    for line in data:
        # If the line contains meta information.
        if line.startswith('#CHROM'):
            # Split the line by tabs.
            meta_info = line.split()
            # Break the loop.
            break

In [22]:
# Load the tgp meta information.
_, tgp_info = load_tgp_meta_data()

In [23]:
# Check that the samples are in extactly the same order.
np.array_equal(np.array(meta_info[9:]), tgp_info['sample'])

True

## Filtering the Original Chromosome 6 VCF File

After running `qc_vcf.py` I will summarize the sites that failed and passed qc.

In [34]:
# Load the qc table.
tgp_org_qc_df, tgp_org_qc_info = load_qc_table('tgp_original')

In [35]:
# Show the head of the dataframe.
tgp_org_qc_df.head()

,chrom,pos,ref,alt,qc_id
0,6,63854,T,G,0
1,6,63979,C,T,0
2,6,63980,A,G,0
3,6,73938,A,G,0
4,6,86820,CT,C,2


In [37]:
# Intialize masks.
is_qced = tgp_org_qc_info['qc_id'] == 0
is_dup = tgp_org_qc_info['qc_id'] == 1
is_multi = tgp_org_qc_info['qc_id'] == 2
# Print a sumary.
print('##***## Original TGP VCF ##***##')
print(f'Total Sites: {tgp_org_qc_df.shape[0]}')
print(f'Duplicated Sites: {is_dup.sum()}')
print(f'Multi-allelic Sites: {is_multi.sum()}')
print(f'Failed QC Sites: {is_dup.sum() + is_multi.sum()}')
print(f'Passed QC Sites: {is_qced.sum()}')

##***## Original TGP VCF ##***##
Total Sites: 5024119
Duplicated Sites: 9781
Multi-allelic Sites: 218430
Failed QC Sites: 228211
Passed QC Sites: 4795908


## Validating `HLA_PBD_CDS` Tables

After running `compile_hla_pbd_cds_vcfs.py` I will validate that the samples are in the correct order and summarize the sites that failed and passed qc.

In [141]:
# Load the tgp meta information.
_, tgp_info = load_tgp_meta_data()
# Load the all sites hla pbd cds table.
hla_pbd_cds_df = load_hla_pbd_cds_vcf('tgp_all_sites')

In [142]:
# Check that the samples are in extactly the same order.
np.array_equal(hla_pbd_cds_df.columns.values[9:], tgp_info['sample'])

True

In [143]:
# Show the head of the dataframe.
hla_pbd_cds_df.head()

#CHROM       POS ID REF ALT QUAL FILTER      INFO FORMAT HG00096 HG00097  \
0       6  29910534  .   G   T    .   PASS  VT=HLA_A     GT     0/0     0/0   
1       6  29910535  .   C   .    .   PASS  VT=HLA_A     GT     0/0     0/0   
2       6  29910536  .   T   C    .   PASS  VT=HLA_A     GT     0/0     0/0   
3       6  29910537  .   C   .    .   PASS  VT=HLA_A     GT     0/0     0/0   
4       6  29910538  .   C   T    .   PASS  VT=HLA_A     GT     0/0     0/0   

  HG00099 HG00100 HG00101 HG00102 HG00103 HG00105 HG00106 HG00107 HG00108  \
0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
1     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
2     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
3     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
4     0/0     0/0     0/0     0/0     1/0     1/0     0/0     0/0     0/0   

  HG00109 HG00110 HG00111 HG00112 HG00113 HG00114 HG00115 HG00116 HG00117  \
0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
1     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
2     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
3     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
4     0/1     1/0     0/0     1/0     1/0     0/0     0/0     0/0     0/0   

  HG00118 HG00119 HG00120 HG00121 HG00122 HG00123 HG00125 HG00126 HG00127  \
0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
1     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
2     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
3     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
4     1/0     0/0     0/1     0/0     0/1     0/0     1/0     0/0     0/0   

  HG00128 HG00129 HG00130 HG00131 HG00132 HG00133 HG00136 HG00137 HG00138  \
0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
1     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
2     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
3     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
4     1/1     0/1     0/0     0/0     1/0     1/0     1/1     0/1     1/0   

  HG00139 HG00140 HG00141 HG00142 HG00143 HG00145 HG00146 HG00148 HG00149  \
0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
1     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
2     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
3     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
4     1/0     0/0     0/0     1/0     1/1     0/0     0/0     1/0     1/1   

  HG00150 HG00151 HG00154 HG00155 HG00157 HG00158 HG00159 HG00160 HG00171  \
0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
1     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
2     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
3     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
4     1/0     0/0     1/0     1/0     1/0     1/0     1/0     1/0     1/1   

  HG00173 HG00174 HG00176 HG00177 HG00178 HG00179 HG00180 HG00181 HG00182  \
0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
1     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
2     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
3     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
4     1/0     1/0     0/1     1/1     0/0     0/0     1/0     1/0     1/0   

  HG00183 HG00185 HG00186 HG00187 HG00188 HG00189 HG00190 HG00231 HG00232  \
0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
1     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0     0/0   
2     0/0     0/0     0/0     0/0     0/0     0/0     0/

In [146]:
# Extract the alternative allele column.
alts = hla_pbd_cds_df.ALT.values
# Intialize masks.
is_mono = alts == '.'
is_bi = np.array([len(alt) == 1 for alt in alts]) & ~is_mono
is_tri = np.array([alt.count(',') == 1 for alt in alts])
is_quad = np.array([alt.count(',') == 2 for alt in alts])
# Intialize a dictionary.
allele_dicc = {
    'mono_allelic': is_mono, 'bi_allelic': is_bi,
    'tri_allelic': is_tri, 'quad_allelic': is_quad,
}

In [147]:
# Extract the locus columns.
loci = hla_pbd_cds_df.INFO.values
# Intialize a dictionary.
locus_dicc = {
    'hla_a': loci == 'VT=HLA_A', 'hla_b': loci == 'VT=HLA_B', 'hla_c': loci == 'VT=HLA_C',
    'hla_dqb1': loci == 'VT=HLA_DQB1', 'hla_drb1': loci == 'VT=HLA_DRB1',
}

In [149]:
# Intialize a dictionary.
hla_pbd_cds_summary = {
    'locus': [], 'n_mono_allelic': [],
    'n_bi_allelic': [], 'n_tri_allelic': [], 'n_quad_allelic': [],
}
# For every hla locus.
for locus in locus_dicc:
    # Update the dictionary.
    hla_pbd_cds_summary['locus'].append(locus)
    # For every allelic type.
    for allele in allele_dicc:
        # Update the dictionary.
        hla_pbd_cds_summary[f'n_{allele}'].append((locus_dicc[locus] & allele_dicc[allele]).sum())
# Convert the dictionary to a dataframe.
hla_pbd_cds_summary_df = pd.DataFrame(hla_pbd_cds_summary)
# Show the dataframe.
hla_pbd_cds_summary_df

,locus,n_mono_allelic,n_bi_allelic,n_tri_allelic,n_quad_allelic
0,hla_a,456,73,15,2
1,hla_b,445,82,16,3
2,hla_c,486,49,9,2
3,hla_dqb1,210,51,8,1
4,hla_drb1,205,44,15,6


In [156]:
# Load the original qc report.
tgp_org_qc_df, tgp_org_qc_info = load_qc_table('tgp_original')
# Intialize a dictionary.
org_qc_dicc = {
    'bi_allelic': tgp_org_qc_info['qc_id'] == 0,
    'dup_record': tgp_org_qc_info['qc_id'] == 1,
    'multi_allelic': tgp_org_qc_info['qc_id'] == 2,
}

In [151]:
# Show the head of the dataframe.
tgp_org_qc_df.head()

,chrom,pos,ref,alt,qc_id
0,6,63854,T,G,0
1,6,63979,C,T,0
2,6,63980,A,G,0
3,6,73938,A,G,0
4,6,86820,CT,C,2


In [152]:
# Extract the positions in the hla pbd cds dataframe.
hla_pbd_cds_pos = hla_pbd_cds_df.POS.values
# Determine the sites that will be replaced.
hla_pbd_cds_is_same = np.isin(hla_pbd_cds_pos, tgp_org_qc_info['pos'])
org_is_same = np.isin(tgp_org_qc_info['pos'], hla_pbd_cds_pos)

In [157]:
# For every qc type.
for qc in org_qc_dicc:
    # Print how many will be replaced.
    print(f'n {qc} replaced: {(org_qc_dicc[qc] & org_is_same).sum()}')

n bi_allelic replaced: 245
n dup_record replaced: 0
n multi_allelic replaced: 44


In [158]:
# Intialize a dictionary.
replace_summary = {
    'locus': [], 'org_type': [], 'n_mono_allelic': [],
    'n_bi_allelic': [], 'n_tri_allelic': [], 'n_quad_allelic': [],
}
# For every hla locus.
for locus in locus_dicc:
    # For every original qc type.
    for qc in ['bi_allelic', 'multi_allelic']:
        # Extract the positions that will be replaced.
        replaced_pos = tgp_org_qc_info['pos'][(org_qc_dicc[qc] & org_is_same)]
        # Intialize the mask.
        is_replaced = np.isin(hla_pbd_cds_pos, replaced_pos)
        # Update the dictionary.
        replace_summary['locus'].append(locus)
        replace_summary['org_type'].append(qc)
        # For every allelic type.
        for allele in allele_dicc:
            # Update the dictionary.
            replace_summary[f'n_{allele}'].append((locus_dicc[locus] & allele_dicc[allele] & is_replaced).sum())
# Convert the dictionary to a dataframe.
replace_summary_df = pd.DataFrame(replace_summary)
# Show the dataframe.
replace_summary_df

,locus,org_type,n_mono_allelic,n_bi_allelic,n_tri_allelic,n_quad_allelic
0,hla_a,bi_allelic,6,59,2,0
1,hla_a,multi_allelic,1,0,12,2
2,hla_b,bi_allelic,4,65,3,0
3,hla_b,multi_allelic,3,3,12,3
4,hla_c,bi_allelic,10,35,3,1
5,hla_c,multi_allelic,0,1,6,1
6,hla_dqb1,bi_allelic,1,32,0,0
7,hla_dqb1,multi_allelic,0,0,0,0
8,hla_drb1,bi_allelic,0,20,4,0
9,hla_drb1,multi_allelic,0,0,0,0


In [166]:
# Intialize a dictionary.
introduce_summary = {
    'locus': [], 'n_mono_allelic': [],
    'n_bi_allelic': [], 'n_tri_allelic': [], 'n_quad_allelic': [],
}
# For every hla locus.
for locus in locus_dicc:
    # Update the dictionary.
    introduce_summary['locus'].append(locus)
    # For every allelic type.
    for allele in allele_dicc:
        # Update the dictionary.
        introduce_summary[f'n_{allele}'].append((locus_dicc[locus] & allele_dicc[allele] & ~hla_pbd_cds_is_same).sum())
# Convert the dictionary to a dataframe.
introduce_summary_df = pd.DataFrame(introduce_summary)
# Show the dataframe.
introduce_summary_df

,locus,n_mono_allelic,n_bi_allelic,n_tri_allelic,n_quad_allelic
0,hla_a,449,14,1,0
1,hla_b,438,14,1,0
2,hla_c,476,13,0,0
3,hla_dqb1,209,19,8,1
4,hla_drb1,205,24,11,6


## Validating the Final VCF Files

In [16]:
# Intialize a dictionary of pbd exons.
pbd_dicc = {
    'HLA_A_PBD_1': (29910534, 29910803),
    'HLA_A_PBD_2': (29911045, 29911320),
    'HLA_B_PBD_1': (31323944, 31324219),
    'HLA_B_PBD_2': (31324465, 31324734),
    'HLA_C_PBD_1': (31238850, 31239125),
    'HLA_C_PBD_2': (31239376, 31239645),
    'HLA_DRB1_PBD': (32551886, 32552155),
    'HLA_DQB1_PBD': (32632575, 32632844),
}

In [22]:
# Load the qc table.
tgp_org_qc_df, tgp_org_qc_info = load_qc_table('tgp_original')
# Intialize masks.
qc_pos = tgp_org_qc_info['pos']
is_bi = tgp_org_qc_info['qc_id'] == 0
is_dup = tgp_org_qc_info['qc_id'] == 1
is_multi = tgp_org_qc_info['qc_id'] == 2
# Intialize a table for the tgp results.
org_dicc = {
    'PBD_REGION': [], 'LENGTH': [], 'N_MONO': [], 'N_BI': [], 'N_MULTI': [],
}
# For every pbd region.
for pbd, (start, end) in pbd_dicc.items():
    # Compute the length.
    length = end - start + 1
    # Create a mask for the region.
    is_pbd = (start <= qc_pos) & (qc_pos <= end)
    # Determine the number of duplicated sites.
    n_dup = np.unique(qc_pos[is_pbd & is_dup]).size
    # Determine the number of multi-allelic sites.
    n_multi = (is_pbd & is_multi).sum()
    # Determine the number of bi-allelic sites.
    n_bi = (is_pbd & is_bi).sum()
    # Compute the number of mono-allelic sites.
    n_mono = length - (n_dup + n_multi + n_bi)
    # Update the dictionary.
    org_dicc['PBD_REGION'].append(pbd)
    org_dicc['LENGTH'].append(length)
    org_dicc['N_MONO'].append(n_mono)
    org_dicc['N_BI'].append(n_bi)
    org_dicc['N_MULTI'].append(n_multi)
# Convert the dictionary to a dataframe.
org_pbd_summary_df = pd.DataFrame(org_dicc)

In [23]:
# Show the results.
org_pbd_summary_df

,PBD_REGION,LENGTH,N_MONO,N_BI,N_MULTI
0,HLA_A_PBD_1,270,225,39,6
1,HLA_A_PBD_2,276,239,28,9
2,HLA_B_PBD_1,276,240,27,9
3,HLA_B_PBD_2,270,213,45,12
4,HLA_C_PBD_1,276,245,25,6
5,HLA_C_PBD_2,270,244,24,2
6,HLA_DRB1_PBD,270,246,24,0
7,HLA_DQB1_PBD,270,237,33,0


In [27]:
# Load the qc table.
tgp_rep_qc_df, tgp_rep_qc_info = load_qc_table('tgp_replaced')
# Intialize masks.
qc_pos = tgp_rep_qc_info['pos']
is_bi = tgp_rep_qc_info['qc_id'] == 0
is_dup = tgp_rep_qc_info['qc_id'] == 1
is_multi = tgp_rep_qc_info['qc_id'] == 2
# Intialize a table for the tgp results.
rep_dicc = {
    'PBD_REGION': [], 'LENGTH': [], 'N_MONO': [], 'N_BI': [], 'N_MULTI': [],
}
# For every pbd region.
for pbd, (start, end) in pbd_dicc.items():
    # Compute the length.
    length = end - start + 1
    # Create a mask for the region.
    is_pbd = (start <= qc_pos) & (qc_pos <= end)
    # Determine the number of duplicated sites.
    n_dup = np.unique(qc_pos[is_pbd & is_dup]).size
    # Determine the number of multi-allelic sites.
    n_multi = (is_pbd & is_multi).sum()
    # Determine the number of bi-allelic sites.
    n_bi = (is_pbd & is_bi).sum()
    # Compute the number of mono-allelic sites.
    n_mono = length - (n_dup + n_multi + n_bi)
    # Update the dictionary.
    rep_dicc['PBD_REGION'].append(pbd)
    rep_dicc['LENGTH'].append(length)
    rep_dicc['N_MONO'].append(n_mono)
    rep_dicc['N_BI'].append(n_bi)
    rep_dicc['N_MULTI'].append(n_multi)
# Convert the dictionary to a dataframe.
rep_pbd_summary_df = pd.DataFrame(rep_dicc)

In [28]:
# Show the results.
rep_pbd_summary_df

,PBD_REGION,LENGTH,N_MONO,N_BI,N_MULTI
0,HLA_A_PBD_1,270,219,45,6
1,HLA_A_PBD_2,276,237,28,11
2,HLA_B_PBD_1,276,229,35,12
3,HLA_B_PBD_2,270,216,47,7
4,HLA_C_PBD_1,276,240,26,10
5,HLA_C_PBD_2,270,246,23,1
6,HLA_DRB1_PBD,270,205,44,21
7,HLA_DQB1_PBD,270,210,51,9


In [35]:
# Load the all sites hla pbd cds table.
hla_pbd_cds_df = load_hla_pbd_cds_vcf('tgp_all_sites')
# Extract the alternative allele and position column.
alts = hla_pbd_cds_df.ALT.values
qc_pos = hla_pbd_cds_df.POS.values
# Intialize masks.
is_mono = alts == '.'
is_bi = np.array([len(alt) == 1 for alt in alts]) & ~is_mono
is_tri = np.array([alt.count(',') == 1 for alt in alts])
is_quad = np.array([alt.count(',') == 2 for alt in alts])
is_multi = is_tri | is_quad
# Intialize a table for the tgp results.
cds_dicc = {
    'PBD_REGION': [], 'LENGTH': [], 'N_MONO': [], 'N_BI': [], 'N_MULTI': [],
}
# For every pbd region.
for pbd, (start, end) in pbd_dicc.items():
    # Compute the length.
    length = end - start + 1
    # Create a mask for the region.
    is_pbd = (start <= qc_pos) & (qc_pos <= end)
    # Determine the number of multi-allelic sites.
    n_multi = (is_pbd & is_multi).sum()
    # Determine the number of bi-allelic sites.
    n_bi = (is_pbd & is_bi).sum()
    # Compute the number of mono-allelic sites.
    n_mono = (is_pbd & is_mono).sum()
    # Update the dictionary.
    cds_dicc['PBD_REGION'].append(pbd)
    cds_dicc['LENGTH'].append(length)
    cds_dicc['N_MONO'].append(n_mono)
    cds_dicc['N_BI'].append(n_bi)
    cds_dicc['N_MULTI'].append(n_multi)
# Convert the dictionary to a dataframe.
cds_pbd_summary_df = pd.DataFrame(cds_dicc)

In [36]:
# Show the results.
cds_pbd_summary_df

,PBD_REGION,LENGTH,N_MONO,N_BI,N_MULTI
0,HLA_A_PBD_1,270,219,45,6
1,HLA_A_PBD_2,276,237,28,11
2,HLA_B_PBD_1,276,229,35,12
3,HLA_B_PBD_2,270,216,47,7
4,HLA_C_PBD_1,276,240,26,10
5,HLA_C_PBD_2,270,246,23,1
6,HLA_DRB1_PBD,270,205,44,21
7,HLA_DQB1_PBD,270,210,51,9


In [41]:
# Intialize a dictionary to store the results.
hla_pbd_exons_validation = {
    'DATASET': [], 'LENGTH': [], 'N_MONO': [], 'N_BI': [], 'N_MULTI': [],
}
# For every dataset.
for dataset, df in [
    ('TGP Original VCF', org_pbd_summary_df),
    ('TGP Replaced VCF', rep_pbd_summary_df),
    ('TGP PBD CDS Tables', cds_pbd_summary_df),
]:
    # Update the dictionary.
    hla_pbd_exons_validation['DATASET'].append(dataset)
    hla_pbd_exons_validation['LENGTH'].append(df.LENGTH.values.sum())
    hla_pbd_exons_validation['N_MONO'].append(df.N_MONO.values.sum())
    hla_pbd_exons_validation['N_BI'].append(df.N_BI.values.sum())
    hla_pbd_exons_validation['N_MULTI'].append(df.N_MULTI.values.sum())
# Convert the dictionary to a dataframe.
hla_pbd_exons_validation_df = pd.DataFrame(hla_pbd_exons_validation)
# Export the validation as a csv.
hla_pbd_exons_validation_df.to_csv('../meta_data/tgp_hla_pbd_exons_validation.csv', index=False)

In [42]:
# Show the results.
hla_pbd_exons_validation_df

,DATASET,LENGTH,N_MONO,N_BI,N_MULTI
0,TGP Original VCF,2178,1889,245,44
1,TGP Replaced VCF,2178,1802,299,77
2,TGP PBD CDS Tables,2178,1802,299,77


## Scripts

Here are the python scripts located in the `scripts`.

### `qc_vcf.py`

In [ ]:
# Import packages.
import gzip
import sys

### sys.argv[1]: path to the qc table with the suffix txt.gz ###
### sys.argv[2]: path to the vcf.gz file ###


# Define a function to identify what duplicated entires from a single chromosome vcf.
def identify_dups(vcf):
    """
    Returns a set of duplicated positions.
    """
    # Intilailize the previous  position.
    prev_pos = None
    # Inialize a set to store duplicated positions.
    dup_set = set()
    # Read the vcf file.
    with gzip.open(vcf, 'rt') as data:
        # Iterate through every line in the original vcf file.
        for line in data:
            # If the line does not contains meta information.
            if not line.startswith('#'):
                # Split the line by tabs.
                spline = line.split()
                # Grab the current poistion.
                pos = spline[1]
                # If the current position is already known to be duplicated.
                if pos in dup_set:
                    # Update the previous position.
                    prev_pos = pos
                # Else-if the current position is the same as the previous position.
                elif pos == prev_pos:
                    # Add the position to the duplicated set.
                    dup_set.add(pos)
                # Else, the current position is unique.
                else:
                    # Update the previous position.
                    prev_pos = pos
    return dup_set

# Define a function to qc a line from the unfiltered vcf.
def qc_vcf_line(line, dup_set):
    """
    Returns the qc information for an autosomal TGP VCF line.
    
    QC IDs:
    - 0 = passed qc (ie biallelic snp)
    - 1 = duplicate record
    - 2 = multi alleleic
    """
    # Split the line by tabs.
    spline = line.split()
    # Grab the chromosome, position, refernce, and alternative alleles.
    chrom, pos, ref, alt = [spline[i] for i in [0, 1, 3, 4]]
    
    # If the current position has duplicated records.
    if pos in dup_set:
        # Return the qc information.
        return f'{chrom}\t{pos}\t{ref}\t{alt}\t1\n'
    # Else-if the length of refernce and alternative fields is not exactly 2,
    # ie if the site is a not mono- or biallelic snp.
    elif (len(ref) + len(alt)) != 2:
        # Return the qc information.
        return f'{chrom}\t{pos}\t{ref}\t{alt}\t2\n'
    # Else, the site is a biallelic snp.
    else:
        # Return the qc information.
        return f'{chrom}\t{pos}\t{ref}\t{alt}\t0\n'

# Define a function to parse an a single chromosome vcf file and output a qc table.
def create_qc_table(qc_table, vcf, buffer_size=100_000):
    """
    Creates a txt.gz file with the qc information per site.
    """
    # Determine the duplicated positions.
    dup_set = identify_dups(vcf)
    # Intialize a list to store the the qc information.
    qc_lines = ['chrom\tpos\tref\talt\tqc_id\n']
    
    # Read the vcf file and intilialize the qc table.
    with gzip.open(vcf, 'rt') as vcf_data, \
         gzip.open(qc_table, 'wt') as qc_file:
        # Iterate through every line in the original vcf file.
        for line in vcf_data:
            # If the line does not contains meta information.
            if not line.startswith('#'):
                # Process and append the current line.
                qc_lines.append(qc_vcf_line(line, dup_set))
            # If the number of qc lines exceeds the buffer size.
            if len(qc_lines) >= buffer_size:
                # Write the qc lines to the file.
                qc_file.writelines(qc_lines)
                # Clear the written qc lines.
                qc_lines.clear()

        # If there are still qc lines to be written.
        if qc_lines:
            # Write the remaining qc lines.
            qc_file.writelines(qc_lines)
    return

# QC the vcf file.
create_qc_table(str(sys.argv[1]), str(sys.argv[2]))

### `extract_qc_regions_files.py`

In [ ]:
# Import packages.
import numpy as np
import pandas as pd
import sys

### sys.argv[1]: dataset prefix ###


# Define a function to load the qc table.
def load_qc_table(prefix):
    """
    Returns the vcf qc table as pandas dataframe and dictionary.
    """
    df = pd.read_csv(f'./meta_data/{prefix}_qc_chr6.txt.gz', sep='\t')
    dicc = {col: df[col].values for col in df.columns.values}
    return df, dicc


# Define a function to extract the regions files.
def extract_regions_file(prefix):
    """
    Returns the regions files for the sites that passed and failed qc.
    """
    # Load the qc table.
    org_qc_df, org_qc_info = load_qc_table(f'{prefix.lower()}_original')
    # Intialize the qc'ed mask.
    is_qced = org_qc_info['qc_id'] == 0
    # Export the qc results.
    org_qc_df[is_qced].to_csv(f'./meta_data/{prefix.lower()}_original_passed_qc_chr6.txt.gz', sep='\t', index=False)
    org_qc_df[~is_qced].to_csv(f'./meta_data/{prefix.lower()}_original_failed_qc_chr6.txt.gz', sep='\t', index=False)
    return

# Extract the region files.
extract_regions_file(str(sys.argv[1]))

### `filter_original_vcf_file.py`

In [ ]:
# Import packages.
import gzip
import sys

### sys.argv[1]: path to the passed qc table with the suffix txt.gz ###
### sys.argv[2]: path to the vcf.gz file ###


# Define a function to filter the original vcf file.
def filter_original_vcf(qced_file, vcf, buffer_size=100_000):
    """
    Outputs the filtered VCF file to stdout.
    """
    # Intialize a set to store the qc'ed postions.
    qced_pos = set()
    # Open the qc'ed file.
    with gzip.open(qced_file, 'rt') as qced_data:
        # Skip the header line.
        next(qced_data)
        # For every line in the qc'ed file.
        for line in qced_data:
            # Update the qc'ed positions set.
            qced_pos.add(int(line.split()[1]))
            
    # Intialize a list to store the qc'ed vcf lines.
    vcf_lines = []
    
    # Read the original vcf file.
    with gzip.open(vcf, 'rt') as vcf_data:
        # Iterate through every line in the original vcf file.
        for line in vcf_data:
            # If the line contains meta information.
            if line.startswith('#'):
                # Append the line to the vcf list.
                vcf_lines.append(line)
            # Else-if the current line passed qc.
            elif int(line.split()[1]) in qced_pos:
                # Append the line to the vcf list.
                vcf_lines.append(line)
            
            # If the number of vcf lines exceeds the buffer size.
            if len(vcf_lines) >= buffer_size:
                # Write the vcf lines to the stdout.
                sys.stdout.writelines(vcf_lines)
                # Clear the written vcf lines.
                vcf_lines.clear()

        # If there are still vcf lines to be written.
        if vcf_lines:
            # Write the remaining vcf lines to stdout.
            sys.stdout.writelines(vcf_lines)
    return

# Filter the original vcf file.
filter_original_vcf(str(sys.argv[1]), str(sys.argv[2]))

### `compile_hla_pbd_cds_vcfs.py`

In [ ]:
# Import packages.
import numpy as np
import pandas as pd
import sys

### sys.argv[1]: dataset prefix ###


# Define a function to load the hla pbd cds tables.
def load_hla_pbd_cds_table(prefix, suffix):
    """
    Returns the HLA PBD CDS table as a pandas dataframe.
    """
    return pd.read_csv(f'./hla_pbd_cds_tables/{prefix.upper()}full_HLASeq{suffix.upper()}.csv')

# Define a function to compile and qc the hla pbd cds tables.
def compile_hla_pbd_cds_tables(prefix):
    """
    Returns headerless vcf files as a .txt.gz .
    """
    # Load and sort all of the hla tables.
    hla_df = pd.concat(
        [load_hla_pbd_cds_table(prefix, locus) for locus in ['a', 'b', 'c', 'dqb1', 'drb1']]
    ).sort_values(by=['POS'], ascending=True, ignore_index=True)
    
    # Subset the dataframe to export later.
    vcf_df = hla_df[['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT']].copy()
    # Reformat the info column.
    vcf_df['INFO'] = 'VT=' + vcf_df['INFO']
    
    # Extract the genotypes as a numpy matrix.
    genos = hla_df.to_numpy()[:, 9:]
    # Extract the sample ids.
    samp_ids = hla_df.columns.values[9:]
    # Extract the reference alleles.
    refs = hla_df.REF.values
    # Intialize a list to store the alternative alleles.
    alts = []
    
    # For every position.
    for i, ref in enumerate(refs):
        # Determine the unique alleles.
        alleles = np.unique(genos[i, :])
        # Recode the reference alleles.
        genos[i, :][genos[i, :] == ref] = '0'
        # If there are missing genotypes.
        if 'X' in alleles:
            # Recode the missing alleles.
            genos[i, :][genos[i, :] == 'X'] = '.'
        # Remove the refernce and missing alleles.
        alleles = alleles[(alleles != ref) & (alleles != 'X')]
        # If there are no alleles.
        if alleles.size == 0:
            # Update the alternate allele column.
            alts.append('.')
        # Else.
        else:
            # For every alternative allele.
            for j, alt in enumerate(alleles):
                # Recode the alternative allele.
                genos[i, :][genos[i, :] == alt] = f'{int(j+1)}'
            # Update the alternate allele column.
            alts.append(','.join(alleles))
    
    # Update the alternative allele column.
    vcf_df['ALT'] = alts
    # Create a mask for the mono- and bi-allelic sites.
    is_qced = np.array([len(alt) == 1 for alt in alts])
    
    # For every sample id.
    for i, samp_id in enumerate(samp_ids):
        # If this is the first chromsome.
        if '.' not in samp_id:
            # Update the vcf dataframe.
            vcf_df[samp_id] = [f'{hap1}/{hap2}' for hap1, hap2 in zip(genos[:, i], genos[:, i+1])]
    
    # Export the vcf dataframes.
    vcf_df.to_csv(f'./meta_data/{prefix.lower()}_all_sites_hla_pbd_cds_vcf_no_header.txt.gz', sep='\t', index=False)
    vcf_df[is_qced].to_csv(f'./meta_data/{prefix.lower()}_mono_bi_allelic_sites_hla_pbd_cds_vcf_no_header.txt.gz', sep='\t', index=False)
    vcf_df[~is_qced].to_csv(f'./meta_data/{prefix.lower()}_multi_allelic_sites_hla_pbd_cds_vcf_no_header.txt.gz', sep='\t', index=False)
    return

# Compile the HLA PBD CDS tables.
compile_hla_pbd_cds_tables(str(sys.argv[1]))

### `compile_hla_pbd_cds_qc_tables.py`

In [ ]:
# Import packages.
import numpy as np
import pandas as pd
import sys

### sys.argv[1]: dataset prefix ###


# Define a function to load the qc table.
def load_qc_table(prefix):
    """
    Returns the vcf qc table as pandas dataframe and dictionary.
    """
    df = pd.read_csv(f'./meta_data/{prefix}_qc_chr6.txt.gz', sep='\t')
    dicc = {col: df[col].values for col in df.columns.values}
    return df, dicc

# Define a function to load the hla pbd cds vcf files.
def load_hla_pbd_cds_vcf(prefix):
    """
    Returns the HLA PBD CDS vcf as a pandas dataframe.
    """
    return pd.read_csv(f'./meta_data/{prefix}_hla_pbd_cds_vcf_no_header.txt.gz', sep='\t')

# Define a function to generate the qc tables for the hla pbd cds vcf.
def compile_hla_pbd_cds_qc_tables(prefix):
    """
    Returns the HLA PBD CDS replaced qc tables.
    """
    # Load the all sites hla pbd cds table.
    hla_pbd_cds_df = load_hla_pbd_cds_vcf(f'{prefix.lower()}_all_sites')
    # Extract the poistions, reference, and alternative allele columns.
    hla_pbd_cds_pos = hla_pbd_cds_df.POS.values
    hla_pbd_cds_refs = hla_pbd_cds_df.REF.values
    hla_pbd_cds_alts = hla_pbd_cds_df.ALT.values
    
    # Intialize allelic masks.
    is_mono = hla_pbd_cds_alts == '.'
    is_bi = np.array([len(alt) == 1 for alt in hla_pbd_cds_alts]) & ~is_mono
    is_tri = np.array([alt.count(',') == 1 for alt in hla_pbd_cds_alts])
    is_quad = np.array([alt.count(',') == 2 for alt in hla_pbd_cds_alts])
    
    # Intialize a qc id column.
    hla_pbd_cds_qc_ids = np.full(hla_pbd_cds_pos.size, -1)
    hla_pbd_cds_qc_ids[is_bi] = 0
    hla_pbd_cds_qc_ids[is_tri | is_quad] = 2
    # Intialize a dictionary.
    hla_pbd_cds_qc_dicc = {
        'chrom': np.full(hla_pbd_cds_pos.size, 6),
        'pos': hla_pbd_cds_pos,
        'ref': hla_pbd_cds_refs,
        'alt': hla_pbd_cds_alts,
        'qc_id': hla_pbd_cds_qc_ids,
    }
    
    # Load the original qc report.
    org_qc_df, org_qc_info = load_qc_table(f'{prefix.lower()}_original')
    # Determine the sites that will be replaced.
    org_is_same = np.isin(org_qc_info['pos'], hla_pbd_cds_pos)
    
    # Concatenate the qc dataframes.
    hla_pbd_cds_qc_df = pd.concat(
        [org_qc_df[~org_is_same], pd.DataFrame(hla_pbd_cds_qc_dicc)]
    ).sort_values(by=['pos'], ascending=True, ignore_index=True)
    # Intialize the qc'ed mask.
    is_qced = hla_pbd_cds_qc_df.qc_id.values <= 0
    # Export the qc results.
    hla_pbd_cds_qc_df.to_csv(f'./meta_data/{prefix.lower()}_replaced_qc_chr6.txt.gz', sep='\t', index=False)
    hla_pbd_cds_qc_df[is_qced].to_csv(f'./meta_data/{prefix.lower()}_replaced_passed_qc_chr6.txt.gz', sep='\t', index=False)
    hla_pbd_cds_qc_df[~is_qced].to_csv(f'./meta_data/{prefix.lower()}_replaced_failed_qc_chr6.txt.gz', sep='\t', index=False)
    return

# Generate the hla pbd cds qc tables.
compile_hla_pbd_cds_qc_tables(str(sys.argv[1]))

### `filter_replaced_vcf_file.py`

In [ ]:
# Import packages.
import gzip
import numpy as np
import pandas as pd
import sys

### sys.argv[1]: path to the passed qc table with the suffix txt.gz ###
### sys.argv[1]: path to the mono- and bi-allelic hla pbd cds vcf with the suffix txt.gz ###
### sys.argv[2]: path to the vcf.gz file ###


# Define a function to filter the original vcf file.
def filter_replaced_vcf(qced_file, hla_pbd_cds_vcf, vcf, buffer_size=100_000):
    """
    Outputs the filtered VCF file to stdout.
    """
    # Load the qc'ed file.
    qced_df = pd.read_csv(qced_file, sep='\t')
    # Extract the qc ids and positions.
    qc_ids = qced_df.qc_id.values
    all_qced_pos = qced_df.pos.values
    # Intialize a set to store the qc'ed postions.
    qced_pos = set(all_qced_pos[qc_ids == 0])
    
    # Load the hla pbd cds vcf.
    hla_pbd_cds_df = pd.read_csv(hla_pbd_cds_vcf, sep='\t')
    # Extract the alternative alleles.
    hla_pbd_cds_alts = hla_pbd_cds_df.ALT.values
    # Intialize allelic masks.
    is_mono = hla_pbd_cds_alts == '.'
    is_bi = np.array([len(alt) == 1 for alt in hla_pbd_cds_alts]) & ~is_mono
    # Filter the hla pbd cds vcf only for bi-allelic sites.
    hla_pbd_cds_vcf_df = hla_pbd_cds_df[is_bi].reset_index(drop=True)
    # Extract the hla pbd cds positions.
    hla_pbd_cds_pos = hla_pbd_cds_vcf_df.POS.values
    
    # Intialize a list to store the qc'ed vcf lines.
    vcf_lines = []
    
    # Read the original vcf file.
    with gzip.open(vcf, 'rt') as vcf_data:
        # Iterate through every line in the original vcf file.
        for line in vcf_data:
            # If the line contains meta information.
            if line.startswith('#'):
                # Append the line to the vcf list.
                vcf_lines.append(line)
            # Else-if the current position is a new or replaced hla pbd cds call.
            elif int(line.split()[1]) in hla_pbd_cds_pos:
                # Append the line to the vcf list.
                vcf_lines.append('\t'.join(hla_pbd_cds_vcf_df[hla_pbd_cds_pos == int(line.split()[1])].values[0].astype(str))+'\n')
            # Else-if the current line passed qc.
            elif int(line.split()[1]) in qced_pos:
                # Append the line to the vcf list.
                vcf_lines.append(line)
            
            # If the number of vcf lines exceeds the buffer size.
            if len(vcf_lines) >= buffer_size:
                # Write the vcf lines to the stdout.
                sys.stdout.writelines(vcf_lines)
                # Clear the written vcf lines.
                vcf_lines.clear()

        # If there are still vcf lines to be written.
        if vcf_lines:
            # Write the remaining vcf lines to stdout.
            sys.stdout.writelines(vcf_lines)
    return

# Filter the replaced vcf file.
filter_replaced_vcf(str(sys.argv[1]), str(sys.argv[2]), str(sys.argv[3]))

### `compute_effective_sequence_lengths.py`

In [ ]:
# Import packages.
import numpy as np
import pandas as pd
import sys

### sys.argv[1]: dataset prefix ###


# Define a function to load the qc table.
def load_qc_table(prefix):
    """
    Returns the vcf qc table as pandas dataframe and dictionary.
    """
    df = pd.read_csv(f'./meta_data/{prefix}_qc_chr6.txt.gz', sep='\t')
    dicc = {col: df[col].values for col in df.columns.values}
    return df, dicc

# Define a function to compute effective sequence lengths.
def compute_eff_seq_lens(prefix):
    """
    Returns the effective sequence lengths for the exons and focal regions as .csv.gz .
    """
    # Load the exons and focal regions.
    exons_df = pd.read_csv(
        './meta_data/ncbi_refseq_select_chr6_exons.txt',
        usecols=[1, 2], names=['start', 'end'], sep='\t'
    )
    regions_df = pd.read_csv(
        './meta_data/chr6_focal_regions.txt',
        names=['region', 'start', 'end'], sep='\t'
    )
    # Compute the length of each exon and region.
    exons_df['length'] = exons_df['end'] - exons_df['start'] + 1
    regions_df['length'] = regions_df['end'] - regions_df['start'] + 1

    # Load the failed qc information.
    _, org_qc_info = load_qc_table(f'{prefix.lower()}_original_failed')
    _, rep_qc_info = load_qc_table(f'{prefix.lower()}_replaced_failed')
    # Extract the unique positions (accounts for duplicated records).
    org_qc_failed_pos = np.unique(org_qc_info['pos'])
    rep_qc_failed_pos = np.unique(rep_qc_info['pos'])
    # Intialize matricies to store the effective sequence lengths.
    exons_esl = np.empty((exons_df.shape[0], 2), dtype=int)
    regions_esl = np.empty((regions_df.shape[0], 2), dtype=int)

    # Iterate through the exons.
    for i, row in exons_df.iterrows():
        # Extracts the coordinates and length.
        start, end, length = row.start, row.end, row.length
        # Intialize masks.
        org_n_failed = ((start <= org_qc_failed_pos) & (org_qc_failed_pos <= end)).sum()
        rep_n_failed = ((start <= rep_qc_failed_pos) & (rep_qc_failed_pos <= end)).sum()
        # Update the matrix.
        exons_esl[i, :] = np.array([length - org_n_failed, length - rep_n_failed])

    # Iterate through the regions.
    for i, row in regions_df.iterrows():
        # Extracts the coordinates and length.
        start, end, length = row.start, row.end, row.length
        # Intialize masks.
        org_n_failed = ((start <= org_qc_failed_pos) & (org_qc_failed_pos <= end)).sum()
        rep_n_failed = ((start <= rep_qc_failed_pos) & (rep_qc_failed_pos <= end)).sum()
        # Update the matrix.
        regions_esl[i, :] = np.array([length - org_n_failed, length - rep_n_failed])

    # Update the dataframes.
    exons_df['original_eff_seq_len'] = exons_esl[:, 0]
    exons_df['replaced_eff_seq_len'] = exons_esl[:, 1]
    regions_df['original_eff_seq_len'] = regions_esl[:, 0]
    regions_df['replaced_eff_seq_len'] = regions_esl[:, 1]
    # Export.
    exons_df.to_csv(f'./meta_data/{prefix.lower()}_ncbi_refseq_select_chr6_exons_eff_seq_lens.csv.gz', index=False)
    regions_df.to_csv(f'./meta_data/{prefix.lower()}_chr6_focal_regions_eff_seq_lens.csv.gz', index=False)
    return

# Compute the effective sequence lengths.
compute_eff_seq_lens(str(sys.argv[1]))